In [6]:
from bertopic import BERTopic
from sklearn.datasets import fetch_20newsgroups 
import torch 
import pandas as pd
import numpy as np

In [21]:
#### Returns a tuple the first element is document_to_topic_tensor
#### The second element is topic_to_word_tensor 
#### Use the nr_topics_1 argument to specify the number of topics that should be present finally. If this parameter is too low
#### take a lot of time.
#### Use the words_per_topic argument to specify the number of representative words per topic. 

def generate_matrices(X,word_to_index, words_per_topic = 10,nr_topics_1 = None): 
    ### get target topics and probabilities of each document topic pair
    model = BERTopic(calculate_probabilities  = True , top_n_words = 10,nr_topics = nr_topics_1) 
    topics,probs = model.fit_transform(X) 
    
    ### calclate remaining topic probability as 1 - sum of given topic probabilities.
    document_to_topic_bas = probs  
    document_to_topic = [] 
    for i in range(len(document_to_topic_bas)): 
        sum1 = 0.0 
        for j in document_to_topic_bas[i]: 
            sum1 = sum1 + j 
        templ = [max(1-sum1,0.0)] 
        for j in document_to_topic_bas[i]: 
            templ.append(j) 
        document_to_topic.append(templ) 
    
    
    
    #print(topics[0] , document_to_topic[0])  
    
    vocab_len = len(list(word_to_index.items())) 
    
    topic_to_word_dict  = model.get_topics()
    topic_len = len(list(topic_to_word_dict.items())) 
  
    ## initialize topic_to_word matrix
    topic_to_word = [] 
    for i in range(topic_len):  
        temparr = []
        for j in range(vocab_len): 
            temparr.append(0) 
        topic_to_word.append(temparr) 
    
    
    for i in topic_to_word_dict: 
        temparr = topic_to_word_dict[i] 
        size = len(temparr) 
        for j in range(size): 
            if(temparr[j][0] in word_to_index): 
                 ## make sure that -1 corresponds to 0
                topic_to_word[i+1][word_to_index[temparr[j][0]]] = temparr[j][1]  
    
    ## convert to pytorch tensors.
    document_to_topic_tens = torch.tensor(document_to_topic) 
    topic_to_word_tens= torch.tensor(topic_to_word) 
    
    return (document_to_topic_tens,topic_to_word_tens) 
            


In [7]:
docs = pd.read_csv('FNC_Bin_Train.csv')



In [12]:
print(docs.columns) 
newser = docs['articleBody'] + docs['Headline'] 
print(newser.values[0])

Index(['Body ID', 'articleBody', 'Headline', 'Stance'], dtype='object')
Recently, a second-grader from Kohl Isilkulsky District accidentally discovered his ability superhero. Seven-year student may be attracted to metal objects and even move them from a distance.

About supernormal schoolboy entire Omsk region learned from the report "12 channel" .

Nicholas Kruglyachenko schoolboy from Isilkul:

 - When I do not even want, everything appears. Once I even pulled glass. He moved on its own. Practically I think to become a superhero.

 

Touching the boy may even affect others and pass them superpowers. Second graders can influence others so that people at the time turned into magnets. 

---

Photo: gtrk-omsk.ruSchoolboy almost killed by electric shock claims he now has superpowers like Magneto


In [3]:
def get_word_pos():  
    word_to_index = dict() 
    count0 = 0
    with open('glove.6B.100d.txt' , 'r',encoding = 'utf-8') as f1: 
        for line in f1: 
            lis = line.split() 
            word_to_index[lis[0]] = count0 
            count0 = count0+1 
    return word_to_index

In [4]:
imp_dict= get_word_pos() 


In [14]:
print(type(list(imp_dict.items()))) 
i_l = list(imp_dict.items()) 
print(i_l[1])  
print(len(i_l))

<class 'list'>
(',', 1)
400000


In [22]:
X,Y = generate_matrices(newser.values[:1500],imp_dict,nr_topics_1 = 20) 

In [17]:
print(X)

tensor([[ 3.8936e-01,  2.1464e-02,  2.2991e-01,  ...,  1.0506e-02,
          2.9568e-02,  1.2574e-02],
        [ 0.0000e+00, 1.6952e-308, 1.2593e-308,  ...,  1.0000e+00,
         1.2909e-308, 8.8370e-309],
        [ 3.4745e-02,  1.8563e-02,  3.0176e-02,  ...,  7.0073e-03,
          2.7917e-02,  8.1383e-03],
        ...,
        [ 0.0000e+00, 8.1445e-308, 2.4800e-308,  ..., 1.9953e-308,
         2.2728e-308, 1.1727e-308],
        [ 8.8858e-03,  1.5717e-03,  1.7658e-03,  ...,  9.7891e-04,
          2.8310e-03,  1.0006e-03],
        [ 0.0000e+00, 2.3957e-308, 2.9623e-308,  ..., 1.9622e-308,
         2.4439e-308, 1.2282e-308]], dtype=torch.float64)


In [23]:
print(X.size()) 
print(Y.size())

torch.Size([1500, 21])
torch.Size([21, 400000])
